In [1]:
import os
import sys
import torch
import speechbrain as sb
from hyperpyyaml import load_hyperpyyaml

/home/tuyendv/miniconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
torchvision is not available - cannot save figures


In [2]:
HPARAM_FILE = "hparams/train.yaml"
DATA_FOLDER = "data/TIMIT"
argv = [
        HPARAM_FILE,
        "--data_folder", DATA_FOLDER,
        ]

In [3]:
hparams_file, run_opts, overrides = sb.parse_arguments(argv)
with open(hparams_file) as fin:
    hparams = load_hyperpyyaml(fin, overrides)

In [4]:
from train import dataio_prep, AlignBrain

train_data, valid_data, test_data, label_encoder = dataio_prep(hparams)

In [5]:
align_brain = AlignBrain(
        modules=hparams["modules"],
        opt_class=hparams["opt_class"],
        hparams=hparams,
        run_opts=run_opts,
        checkpointer=hparams["checkpointer"],
    )
align_brain.label_encoder = label_encoder

In [6]:
sample = train_data[0]
sample

/data/codes/sb-apa/aligner/speechbrain/dataio/encoder.py:721: UserWarning: TextEncoder.expect_len was never called: assuming category count of 61 to be correct! Sanity check your encoder using `.expect_len`. Ensure that downstream code also uses the correct size. If you are sure this does not apply to you, use `.ignore_len`.
  warnings.warn(


{'id': 'MEJS0_SX70.WAV',
 'sig': tensor([-1.5259e-04,  6.1035e-05,  6.1035e-05,  ..., -1.2207e-04,
         -3.0518e-05, -9.1553e-05]),
 'phn_encoded': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0]),
 'phn_ends': tensor([ 2040.,  2760.,  3640.,  4280.,  5138.,  6767.,  8520.,  9274., 10440.,
         12360., 13640., 14640.])}

In [28]:
ckpt_dir = "/data/codes/sb-apa/aligner/results/augment_noise_CRDNN/1986/save/CKPT+2024-02-04+11-48-32+00"

model_ckpt_path = f'{ckpt_dir}/model.ckpt'
output_ckpt_path = f'{ckpt_dir}/output.ckpt'

model_state_dict = torch.load(model_ckpt_path)
out_state_dict = torch.load(output_ckpt_path)

align_brain.modules.model.load_state_dict(model_state_dict)
align_brain.modules.output.load_state_dict(out_state_dict)

<All keys matched successfully>

In [29]:
from speechbrain.dataio.dataloader import make_dataloader

dataloader = make_dataloader(train_data)

In [30]:
for batch in dataloader:
    break

phns, phn_lens = batch.phn_encoded
wavs, wav_lens = batch.sig
phn_ends, _ = batch.phn_ends

In [31]:
alignments = align_brain.infer(wavs, wav_lens, phns, phn_lens)

In [47]:
wavs.shape[1] / len(alignments[0])

159.17391304347825

In [44]:
print(alignments)

[[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 4, 5, 5, 6, 6, 6, 7, 8, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 20, 20, 21, 22, 23, 24, 25, 25, 25, 25, 26, 27, 27, 27, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 30, 30, 31, 32, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2]]


In [42]:
phn_ends

tensor([[ 2040.,  2760.,  3640.,  4280.,  5138.,  6767.,  8520.,  9274., 10440.,
         12360., 13640., 14640.]])

In [32]:
align_brain.modules.aligner.calc_accuracy(alignments, phn_ends, phns)

tensor([89.3238])